In [19]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
from sklearn.metrics import accuracy_score
from sklearn import model_selection

In [20]:
df = pd.read_csv('data/train.csv')
df = df.dropna()
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,total_acc,initial_list_status,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,record_id
3,4200.0,36 months,13.33,142.19,C,C3,Receptionist,< 1 year,MORTGAGE,21600.0,...,19.0,w,0.0,1.0,INDIVIDUAL,0.0,165.0,28187.0,14500.0,453447199
7,14000.0,36 months,15.31,487.45,C,C2,Department of the Army,10+ years,MORTGAGE,96000.0,...,29.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,276267.0,39700.0,453322484
12,20000.0,36 months,11.99,664.20,B,B3,"Sr. Specialist, Tax",10+ years,RENT,110000.0,...,35.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,41806.0,54600.0,453423047
16,8650.0,36 months,12.12,287.80,B,B3,Haven Hospice,8 years,RENT,76000.0,...,20.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,614403.0,10000.0,453286318
17,25000.0,60 months,17.27,624.95,C,C5,Cheney Public Schools,6 years,MORTGAGE,108000.0,...,40.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,337866.0,23800.0,453277374


In [21]:
df.replace('n/a', np.nan,inplace=True)
df.emp_length.fillna(value=0,inplace=True)

df['emp_length'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df['emp_length'] = df['emp_length'].astype(int)

data = df.drop(['emp_title', 'home_ownership', 'pymnt_plan','record_id', 'earliest_cr_line', 'grade', 'sub_grade', 'application_type', 'issue_d', 'zip_code', 'policy_code', 'verification_status', 'purpose', 'initial_list_status', 'addr_state'], axis=1)
data['term'] = pd.factorize(data['term'])[0]
data['loan_amnt'] = data['loan_amnt'].astype(float)

In [22]:
data.describe()

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
count,69010.000000,69010.000000,69010.000000,69010.000000,69010.000000,6.901000e+04,69010.000000,69010.000000,69010.000000,69010.000000,...,69010.000000,69010.000000,69010.000000,69010.000000,69010.000000,69010.000000,69010.000000,69010.000000,6.901000e+04,6.901000e+04
mean,14292.887987,0.249544,14.782499,443.981473,6.301927,7.807545e+04,0.759499,17.190718,0.605434,0.894769,...,11.732676,0.182727,14109.394870,54.866966,28.294088,0.014795,0.009144,259.912650,1.482840e+05,2.651965e+04
std,8189.266899,0.432752,4.311040,248.283839,3.493238,5.951871e+04,0.427391,7.846553,1.081644,1.090281,...,4.973530,0.516418,17293.240759,23.347583,11.516417,0.132956,0.104336,2328.447697,1.577138e+05,2.586940e+04
min,1000.000000,0.000000,5.320000,30.420000,1.000000,8.000000e+03,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000e+02
25%,8000.000000,0.000000,11.990000,267.060000,3.000000,5.000000e+04,1.000000,11.330000,0.000000,0.000000,...,8.000000,0.000000,5687.000000,37.900000,20.000000,0.000000,0.000000,0.000000,3.198625e+04,1.210000e+04
50%,12000.000000,0.000000,14.330000,388.110000,7.000000,6.800000e+04,1.000000,16.680000,0.000000,1.000000,...,11.000000,0.000000,10189.000000,55.700000,27.000000,0.000000,0.000000,0.000000,9.570600e+04,2.010000e+04
75%,20000.000000,0.000000,17.570000,576.290000,10.000000,9.250000e+04,1.000000,22.660000,1.000000,1.000000,...,14.000000,0.000000,17544.000000,72.700000,35.000000,0.000000,0.000000,0.000000,2.221028e+05,3.300000e+04
max,35000.000000,1.000000,28.990000,1424.570000,10.000000,7.141778e+06,1.000000,39.990000,22.000000,6.000000,...,55.000000,12.000000,975800.000000,892.300000,111.000000,5.000000,5.000000,296368.000000,8.000078e+06,1.013700e+06


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69010 entries, 3 to 200188
Data columns (total 21 columns):
loan_amnt                     69010 non-null float64
term                          69010 non-null int64
int_rate                      69010 non-null float64
installment                   69010 non-null float64
emp_length                    69010 non-null int32
annual_inc                    69010 non-null float64
loan_status                   69010 non-null int64
dti                           69010 non-null float64
delinq_2yrs                   69010 non-null float64
inq_last_6mths                69010 non-null float64
mths_since_last_delinq        69010 non-null float64
open_acc                      69010 non-null float64
pub_rec                       69010 non-null float64
revol_bal                     69010 non-null float64
revol_util                    69010 non-null float64
total_acc                     69010 non-null float64
collections_12_mths_ex_med    69010 non-null flo

In [33]:
array = data.values
Y = data['loan_status'].values

X = data.drop(['loan_status'], axis=1)

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.2, random_state=7)

        loan_amnt  term  int_rate  installment  emp_length  annual_inc    dti  \
41516     11775.0     0     16.29       415.67          10     52000.0  18.05   
105242    10000.0     0     11.44       329.48          10     67000.0  21.91   
199377    29700.0     0     15.31      1034.08           4    100000.0  20.98   
196656    13500.0     0     14.99       467.92          10     58000.0  22.70   
143140    16000.0     0     15.31       557.08           6     46000.0  27.00   
114310     4800.0     0     19.22       176.49          10     74000.0  24.51   
33273     12000.0     1     17.57       301.93           1    105000.0  19.77   
182287    15000.0     0      9.99       483.94           9     76378.0  20.35   
120715     2500.0     0     16.99        89.12           1     15000.0  19.44   
69253      3000.0     0     24.08       117.83           3     52000.0  13.63   
142465     9200.0     0     12.49       307.73           1     82700.0   5.78   
151146     7200.0     0     

In [34]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [35]:
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [39]:
print(x_test)

[[-0.28368213 -0.57863309 -1.90272457 ... -0.13025348  0.59600237
   1.22735746]
 [ 0.44878711 -0.57863309 -0.62803526 ... -0.13025348 -0.20873724
   0.78597966]
 [ 0.69294352 -0.57863309 -1.61685675 ... -0.13025348  1.222561
   0.77874396]
 ...
 [ 1.22703568  1.72821089  1.48785526 ... -0.13025348 -0.53011516
  -0.39705756]
 [-0.28978604 -0.57863309 -0.26718572 ...  5.68472036  1.14718986
  -0.02441892]
 [ 0.49151448 -0.57863309  0.51309285 ... -0.0574686  -0.63084151
  -0.75160694]]


In [36]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [45]:
classifier.fit(x_test,y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [37]:
y_pred = classifier.predict(x_test)

In [40]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [41]:
cm

array([[  297,  2974],
       [  286, 10245]], dtype=int64)

In [42]:
from sklearn import metrics
print (metrics.accuracy_score(y_test,classifier.predict(x_test)))

0.7638023474858716


In [46]:
from sklearn.linear_model import Lasso, Ridge

In [51]:
lassoregressor = Lasso().fit(X, y)
['{1} : {0}'.format(a, b) for a, b in zip(lassoregressor.coef_, data.columns)]

['loan_amnt : -1.106275013585415e-05',
 'term : -0.0',
 'int_rate : -0.0',
 'installment : 0.0001367892566279195',
 'emp_length : 0.0',
 'annual_inc : 4.2820600882713865e-07',
 'loan_status : -0.0',
 'dti : -0.0',
 'delinq_2yrs : -0.0',
 'inq_last_6mths : 0.0',
 'mths_since_last_delinq : -0.0',
 'open_acc : -0.0',
 'pub_rec : -2.4980161587346575e-06',
 'revol_bal : -0.0',
 'revol_util : 0.0',
 'total_acc : -0.0',
 'collections_12_mths_ex_med : -0.0',
 'acc_now_delinq : 9.716346854953094e-08',
 'tot_coll_amt : 1.849802069682357e-07',
 'tot_cur_bal : 2.047123280817316e-06']

In [52]:
ridgeregressor = Ridge().fit(X, y)
['{1} : {0}'.format(a, b) for a, b in zip(ridgeregressor.coef_, data.columns)]

['loan_amnt : 1.5999643314563168e-06',
 'term : -0.09891925814497053',
 'int_rate : -0.013631638085107508',
 'installment : -0.00013925414303819222',
 'emp_length : 0.004408013146475541',
 'annual_inc : 9.420099133443446e-08',
 'loan_status : -0.0057746700391722375',
 'dti : -0.01282381540766983',
 'delinq_2yrs : -0.004423368809419614',
 'inq_last_6mths : 0.0002455723868396803',
 'mths_since_last_delinq : -0.004212279995268367',
 'open_acc : -0.007904681567281524',
 'pub_rec : -2.1635793480788066e-07',
 'revol_bal : -0.00025224142208904677',
 'revol_util : 0.0026233079093602426',
 'total_acc : -0.02827269714994951',
 'collections_12_mths_ex_med : -0.03167484652278957',
 'acc_now_delinq : -4.1003901385348475e-07',
 'tot_coll_amt : 1.279800182056761e-07',
 'tot_cur_bal : 4.810978721354185e-07']

In [10]:
y = data.loan_status

cols = data.columns
X = pd.get_dummies(data[cols])

from sklearn import preprocessing



le = preprocessing.LabelEncoder()
le.fit(y)

y = le.transform(y)
X = X.as_matrix()

from sklearn import linear_model

logistic = linear_model.LogisticRegression()

logistic.fit(X, y)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)